<a href="https://colab.research.google.com/github/mohansameer1983/NLP/blob/main/MMAI_891_Wysdom_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This cell installs and sets up DistilBert import, as well as the dataset, which we will 
# use tf.datasets to load (https://www.tensorflow.org/datasets/catalog/overview)

!pip install -q transformers tfds-nightly

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import LabelEncoder

try: # this is only working on the 2nd try in colab :)
  from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig, AutoTokenizer, AutoModel, AutoConfig
except Exception as err: # so we catch the error and import it again
  from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig, AutoTokenizer, AutoModel, AutoConfig

import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score

import tensorflow_datasets as tfds

import re

In [2]:
# Mount Google drive so dataset can be accessed
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preparation

In [3]:
def load_data():
  data_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/public_data.csv")
  return data_df

orig_data = load_data()
orig_data

id                                            message  \
0     10480                           start the robotic vacuum   
1      4210          i am sorry but i think you are not right.   
2      2443  what is the time difference between california...   
3      5869                              tell me my list names   
4      2801                 beep when i get an email from john   
...     ...                                                ...   
8879   9708             search for a train ticket to newcastle   
8880   8278               is there a food festival in the area   
8881  10509                       make the lights blue in here   
8882   7046                           play from favorites song   
8883    248                                send a replay email   

                      label  
0              iot_cleaning  
1            general_negate  
2          datetime_convert  
3               lists_query  
4               email_query  
...                     ...  
8879       transport_ticket  
8880  recommendation_events  
8881    iot_hue_lightchange  
8882             play_music  
8883        email_sendemail  

[8884 rows x 3 columns]

In [4]:
orig_data.drop_duplicates(inplace= True)

In [5]:
orig_data.isna().sum()

id         0
message    0
label      0
dtype: int64

In [6]:
max_words = []
for row in orig_data['message']:
    max_words.append(len(row.split()))
print(f"The max word count is {np.asarray(max_words).max()}")

The max word count is 25


#Split Data

## Training Data Split

In [7]:
def extract_text_and_y(df, test_size):

  text = df['message']

  le = LabelEncoder()
  df['label_encoded'] = le.fit_transform(df['label'])
  y = df['label_encoded']

  label_dict = (df[['label','label_encoded']].drop_duplicates()
              .sort_values(by='label_encoded')
              .reset_index(drop=True)['label']
              .to_dict())
  


  X_train, X_val, y_train, y_val = train_test_split(text, y, test_size=test_size, random_state=2)
  return X_train, X_val, y_train, y_val, label_dict

# the following prepares the input for running in DistilBert
train_text, val_text, train_y, val_y, label_dict = extract_text_and_y(orig_data, 0.20)

In [8]:
train_text

8011    save the information which is provided by aman...
2612                       please raise the lights to max
8642                            the song touches my heart
3724                         yeap you get that perfectly.
4753                                     mute immediately
                              ...                        
1099                                         play my game
2514                                     back one episode
6637         i can't hear you can you say it again louder
2575    tell me about current status of sunderland vs ...
7336                                make this my favorite
Name: message, Length: 7107, dtype: object

In [9]:
train_y

8011    38
2612    32
8642    38
3724    16
4753     4
        ..
1099    43
2514    45
6637     5
2575    41
7336    38
Name: label_encoded, Length: 7107, dtype: int64

In [10]:
label_dict

{0: 'alarm_query',
 1: 'alarm_remove',
 2: 'alarm_set',
 3: 'audio_volume_down',
 4: 'audio_volume_mute',
 5: 'audio_volume_up',
 6: 'calendar_query',
 7: 'calendar_remove',
 8: 'calendar_set',
 9: 'cooking_recipe',
 10: 'datetime_convert',
 11: 'datetime_query',
 12: 'email_addcontact',
 13: 'email_query',
 14: 'email_querycontact',
 15: 'email_sendemail',
 16: 'general_affirm',
 17: 'general_commandstop',
 18: 'general_confirm',
 19: 'general_dontcare',
 20: 'general_explain',
 21: 'general_joke',
 22: 'general_negate',
 23: 'general_praise',
 24: 'general_quirky',
 25: 'general_repeat',
 26: 'iot_cleaning',
 27: 'iot_coffee',
 28: 'iot_hue_lightchange',
 29: 'iot_hue_lightdim',
 30: 'iot_hue_lightoff',
 31: 'iot_hue_lighton',
 32: 'iot_hue_lightup',
 33: 'iot_wemo_off',
 34: 'iot_wemo_on',
 35: 'lists_createoradd',
 36: 'lists_query',
 37: 'lists_remove',
 38: 'music_likeness',
 39: 'music_query',
 40: 'music_settings',
 41: 'news_query',
 42: 'play_audiobook',
 43: 'play_game',
 44

### Visualize Data

In [11]:
# Get length of all the messages in the train set
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins = 30)

## Creating Embeddings using DistilBERT 

In [12]:
#Get DistilBERT Tokenizer
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', add_prefix_space=True, use_fast=False)

In [22]:
# Check longest token sequence in entire text
def checkForMaxTokenSeqInData(text_list, tokenizer_i):
  
    max_val = 0
    for sent in text_list:
        try:
            sent_tok_len = len(tokenizer_i.tokenize(sent))
            max_val = sent_tok_len if (sent_tok_len > max_val) else max_val
        except:
            pass
    
    print(f"The maximum amount of tokens in the dataset is {max_val}")
    return max_val

MAX_VAL = checkForMaxTokenSeqInData(train_text.tolist()+val_text.tolist(),tokenizer)

The maximum amount of tokens in the dataset is 28


In [23]:
def encode_text(tokenizer_i, text, max_text_length):

    # Encode the sentence
    encodedText = tokenizer_i.batch_encode_plus(
        text,  # the sentence to be encoded
        add_special_tokens=True,  # Add [CLS] and [SEP]
        max_length = max_text_length,
        padding='max_length',
        truncation=True
        )
    # Get the input IDs and attention mask in tensor format
    input_ids = tf.convert_to_tensor(encodedText['input_ids'])
    attention_mask = tf.convert_to_tensor(encodedText['attention_mask'])
    print(input_ids[0])
    print(tokenizer.convert_ids_to_tokens(input_ids[0]))
    return input_ids, attention_mask

train_input, train_mask = encode_text(tokenizer, train_text, MAX_VAL)
val_input, val_mask = encode_text(tokenizer, val_text, MAX_VAL)

train_model_inputs_and_masks = {
    'inputs' : train_input,
    'masks' : train_mask
}

val_model_inputs_and_masks = {
    'inputs' : val_input,
    'masks' : val_mask
}

tf.Tensor(
[  101  3828  1996  2592  2029  2003  3024  2011 25933  2078  2006  2023
  2299   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0], shape=(28,), dtype=int32)
['[CLS]', 'save', 'the', 'information', 'which', 'is', 'provided', 'by', 'ama', '##n', 'on', 'this', 'song', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tf.Tensor(
[  101  2131  2033  1996  2334  3345 10984  2000  7688 20116 21246   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0], shape=(28,), dtype=int32)
['[CLS]', 'get', 'me', 'the', 'local', 'train', 'timing', 'to', 'destination', 'cs', '##tm', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


# Modelling

## Build and Train Model

In [24]:
def build_model(base_model, trainable=False, params={}):
    inputs = Input(shape = (params['max_seq_length'],),name='input_ids', dtype='int32')
    masks  = Input(shape = (params['max_seq_length'],),name='input_attention', dtype='int32')

    base_model.trainable = trainable

    dbert_output = base_model([inputs, masks])[0]
    dense_layer = Dense(params['dense_layer_1_units'],activation='relu')(dbert_output)
    dropout_layer = Dropout(params['dropout_rate_1'])(dense_layer)
    dense_layer = Dense(params['dense_layer_2_units'],activation='relu')(dropout_layer)
    dropout_layer = Dropout(params['dropout_rate_2'])(dense_layer)
    flatten = Flatten()(dense_layer)
    probs = Dense(len(label_dict), activation='softmax', name="output")(flatten)

    model = keras.Model(inputs=[inputs, masks], outputs=probs)
    model.summary()
    return model

MODEL_NAME= 'distilbert-base-uncased'

# Configure DistilBERT's initialization
config = DistilBertConfig(output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

params={
        "dense_layer_1_units": 128,        
        "dropout_rate_1": 0.2,
	      "dense_layer_2_units": 32,
        "dropout_rate_2": 0.2,
        "regularizer_l2_rate":0.01,
        "max_seq_length":MAX_VAL
        }

model = build_model(dbert_model, params=params)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 28)]         0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 28)]         0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'input_attention[0][0]']        
                                one, 28, 768),                                                    
                                 hidden_states=((No                                         

In [29]:
def compile_model(model, lr):

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.sparse_categorical_crossentropy
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics='accuracy')
    return model


model = compile_model(model, 0.003)

In [27]:
# Add Callbacks

# Model Checkpoint
tl_checkpoint_1 = ModelCheckpoint(filepath = '/content/drive/MyDrive/Colab Notebooks/Models/NLP/distilBert_text_classification_model_weights.hdf5', save_best_only = True, save_weights_only=True, verbose = 0)

# Early Stopping Checkpoint
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [31]:
def train_model(model, model_inputs_and_masks_train, model_inputs_and_masks_val,
    y_train, y_val, batch_size, num_epochs):

    # Training Data - Tensor Conversion
    input_ids_train = model_inputs_and_masks_train['inputs']
    attention_masks_train = model_inputs_and_masks_train['masks']
    labels_train = tf.convert_to_tensor(y_train[:])

    # Validation data - Tensor Conversion
    input_ids_val = model_inputs_and_masks_val['inputs']
    attention_masks_val = model_inputs_and_masks_val['masks']
    labels_val = tf.convert_to_tensor(y_val[:])

    # Preparing Training Dataset
    dataset_train = tf.data.Dataset.from_tensors(( (input_ids_train, attention_masks_train), labels_train ))
    dataset_val = tf.data.Dataset.from_tensors(( (input_ids_val, attention_masks_val), labels_val ))

    history = model.fit(dataset_train,
          epochs=num_epochs,
          validation_data=dataset_val,
          batch_size=batch_size, verbose=1,
          callbacks=[tl_checkpoint_1,EarlyStoppingCallback])
    
    return model, history

model, history = train_model(model, train_model_inputs_and_masks, val_model_inputs_and_masks, train_y, val_y, batch_size=32, num_epochs=100)

Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 0.2187 - accuracy: 0.9311 - val_loss: 0.5551 - val_accuracy: 0.8497
Epoch 2/100
1/1 [==============================] - 5s 5s/step - loss: 0.2091 - accuracy: 0.9365 - val_loss: 0.5547 - val_accuracy: 0.8464
Epoch 3/100
1/1 [==============================] - 4s 4s/step - loss: 0.1897 - accuracy: 0.9446 - val_loss: 0.5626 - val_accuracy: 0.8475
Epoch 4/100
1/1 [==============================] - 4s 4s/step - loss: 0.1942 - accuracy: 0.9403 - val_loss: 0.5559 - val_accuracy: 0.8486
Epoch 5/100
1/1 [==============================] - 4s 4s/step - loss: 0.1912 - accuracy: 0.9415 - val_loss: 0.5561 - val_accuracy: 0.8526
Epoch 6/100
1/1 [==============================] - 4s 4s/step - loss: 0.1874 - accuracy: 0.9437 - val_loss: 0.5550 - val_accuracy: 0.8548
Epoch 7/100
1/1 [==============================] - 4s 4s/step - loss: 0.1857 - accuracy: 0.9444 - val_loss: 0.5592 - val_accuracy: 0.8497
Epoch 8/100
1/1 [=================

# Predict & Calculate Score

In [32]:
# Validation data - Tensor Conversion
def predict_labels(model, model_inputs_and_masks_val, y_vals):
  input_ids_val = model_inputs_and_masks_val['inputs']
  attention_masks_val = model_inputs_and_masks_val['masks']
  labels_val = tf.convert_to_tensor(y_vals[:])
  dataset_val = tf.data.Dataset.from_tensors(( (input_ids_val, attention_masks_val), labels_val ))
  output_val = model.predict(dataset_val)
  predicted_labels = np.argmax(output_val, axis=1)
  return predicted_labels

predicted_labels = predict_labels(model, val_model_inputs_and_masks,val_y)
predicted_labels

array([59, 58,  6, ..., 12, 57, 51])

In [33]:
# Calculate Scores
def calculate_score(val_y, predicted_labels):
  ari_val = adjusted_rand_score(val_y, predicted_labels)
  ami_val = adjusted_mutual_info_score(val_y, predicted_labels, average_method='arithmetic')
  print("ari_val:", ari_val)
  print("ami_val:", ami_val)
  return

calculate_score(val_y, predicted_labels)

ari_val: 0.7445293697674149
ami_val: 0.8256530220334212


#Fine Tuning Custom DistilBERT Model

In [ ]:
# Unfreeze distilBERT layers and make available for training
def fine_tune_transfer_learning_model(base_transfomer_model, custom_model, learning_rate):

    base_transfomer_model.trainable = True
    
    # Recompile model after unfreezing
    model_updated = compile_model(custom_model, learning_rate)
    custom_model.summary()
    
    model_updated, history2 = train_model(model_updated, train_model_inputs_and_masks, val_model_inputs_and_masks, train_y, val_y, batch_size=16, num_epochs=5)

    return custom_model, history2

model2, history2=fine_tune_transfer_learning_model(dbert_model, model, 1e-5)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 28)]         0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 28)]         0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'input_attention[0][0]']        
                                one, 28, 768),                                                    
                                 hidden_states=((No                                         